In [1]:
#imports
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI


In [2]:
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY');

if not api_key:
    print("No API Key loaded")
elif not api_key.startswith("sk-proj-"):
    print("Your API Key does not start with sk-proj. Maybe it was copied wrongly. Please check it")
elif api_key.strip() != api_key:
    print("Your API Key does contain some spaces. Please remove them")
else: 
    print("Your API Key looks good")  

Your API Key looks good


In [3]:
openai = OpenAI(api_key=api_key);

In [4]:
#Verify that the connection with OpenAI is working correctly
message = "Hello ChatGPT, I am calling you using your API. I made it!!!"
response = openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user", "content": message}])
response.choices[0].message.content

"That's great to hear! Congratulations on successfully using the API! If you have any questions or need assistance with anything related to it, feel free to ask."

In [5]:
class Website:
    """
    Utility class to scape webpage html content given the page url
    """
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)    
        
        

In [6]:
santa_clause_webside = Website("https://it.wikipedia.org/wiki/Santa_Clause")
print(santa_clause_webside.title)

Santa Clause - Wikipedia


In [7]:
#System Prompt and User Prompt
system_prompt="You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [8]:
def user_prompt_for(website: Website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows;\n\
    please provide a short summary of this website in markdown. \
    If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [9]:
def messages_for(website: Website):
    return [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt_for(website)}
    ]

In [10]:
#summarize fuction will get everything together
def summarize(url: str) -> str :
    website = Website(url);
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content 


In [11]:
display(Markdown(summarize("https://www.aranzulla.it/come-funziona-sito-semplice-di-tim-1272269.html")))

# Summary of "Come funziona Sito Semplice di TIM" by Salvatore Aranzulla

This website offers a detailed guide on "Sito Semplice," a service provided by TIM Business for creating professional websites, blogs, e-commerce stores, and booking sites without needing professional assistance.

## Key Features:
- **User-Friendly Interface**: Sito Semplice utilizes a drag-and-drop editor that allows users to easily design their websites using an intuitive interface.
- **Templates and Components**: Users have access to over 100 components and more than 1,000 customizable templates created by professional designers.
- **Trial Period**: The service is free for the first three months, with no credit card information required initially.
- **All-Inclusive Pricing**: The offering includes various features under a single package, reducing the complexity of plans.
  
## Plans:
- **Sito Semplice Free**: 
  - Free for the first 3 months
  - Domain of 3rd level, 200 MB of online storage, and some basic applications.
  
- **Sito Semplice Professional**:
  - €10 + VAT per month
  - Custom second-level domain, unlimited storage, advanced site statistics, e-commerce features, and more.

## Additional Services:
Users can create:
- **Blogs** with complete management of articles, categories, and tags.
- **E-commerce** sites with straightforward product and order management.
- **Booking features** for hospitality services.

## Support and Resources:
For further assistance, users can access tutorials on the official Sito Semplice website and contact support through chat or phone for more personalized help.

## About the Author:
Salvatore Aranzulla is a well-known Italian tech blogger and contributor to various tech-related publications, recognized for simplifying complex IT topics.

For more information and to get started, users are encouraged to visit the official Sito Semplice page.